In [1]:
import numpy as np
import pandas as pd
import numpy.linalg as la
from scipy.linalg import solve_triangular
from typing import Callable


### FDIPA

In [2]:
def line_search(f: Callable, df: Callable, g: Callable, x: np.ndarray, eta: float, d: np.ndarray, bar_lambda: np.ndarray, nu: float, m: int):
    """
    Line-Search
    -----------
    Compute t which is the step length, the first number of the sequence {1, v, v2, v3 ,...} that satisfying the following conditions

    Parameters
    ----------
    eta in (0, 1)
    d direction
    bar_lambda lagrangian multiplier
    nu  in (0, 1)

    Return
    ------
    The step length
    """
    t = 1
    while True:
        x_td = x + t * d
        if f(x_td) <= f(x) + t * eta * d.dot(df(x)):
            g_x = g(x)
            g_x_td = g(x_td)
            if all((g_x_td[k] < 0 and bar_lambda[k] >= 0) or (g_x_td[k] <= g_x[k] and bar_lambda[k] < 0) for k in range(m)):
                break
        t = t * nu
    return t


In [3]:
def FDIPA(f: Callable, df: Callable, g: Callable, dg: Callable, x: np.ndarray, tol: float, xi: float, eta: float, phi: float, nu: float, m: int, MAX_ITER=100):
    """
    Feasible Direction Interior-Point
    ---------------------------------
    It is a variant of the interior-point method, which is a popular approach for solving large-scale convex optimization problems.

    Parameters
    ----------
    xi  in (0, 1)
    eta in (0, 1)
    phi > 0
    nu  in (0, 1)
    """
    lam = np.ones(m)
    w = np.ones(m)
    n = len(x)
    iter = 0
    t = 1
    B = np.identity(n)
    Z = np.diag(lam)    # Z[i,i]=lam[i]
    while iter < MAX_ITER:
        g_gradient = dg(x)
        f_gradient = df(x)
        # Step 1 Computation of a search direction
        G = np.diag(g(x))   # G[i,i]=g[i]
        N = np.r_[np.c_[B, g_gradient],
                  np.c_[Z @ g_gradient.T, G]]
        Q, R = la.qr(N)
        # step 1.1 Compute (d_a, lambda_a) by solving the linear system
        z1 = Q.T @ np.r_[-f_gradient,
                         np.zeros(m)]
        x_1 = solve_triangular(R, z1)  # x = [d_a, lam_a]
        d_a = x_1[:n]
        lambda_a = x_1[n:]
        error = la.norm(d_a)
        if error < tol:
            return x
        # step 1.2 Compute (d_b, lambda_b) by solving the linear system
        z2 = Q.T @ np.r_[np.zeros(n),
                         - Z @ w]
        x_2 = solve_triangular(R, z2)
        d_b = x_2[:n]
        lambda_b = x_2[n:]
        # common usage
        a = phi * d_a.dot(d_a)
        # step 1.3
        if (d_bf_grad := d_b.dot(f_gradient)) > 0:
            b = (xi - 1) * d_a.dot(f_gradient) / d_bf_grad
            rho = min(a, b)
        else:
            rho = a
        # step 1.4 Compute the search direction
        d = d_a + rho * d_b
        # and also
        bar_lambda = lambda_a + rho * lambda_b
        # Step 2. Line Search. Compute t, the first number of the sequence {1,v,v^2,v^3,....} satisfying
        t = line_search(f, df, g, x, eta, d, bar_lambda, nu, m)
        x = x + t * d
        iter += 1
        print(f"{iter:2d} {t:1.10f} {x} {f(x):1.9f} {error:1.9f}")


**Example**

\begin{align*}
\min_{\mathbf x}\quad&2x^2+y^2\\
\text{s.t}\quad&4x^2+y^2-2 \le 0\\ 
&4x+y+1\le0
\end{align*}

In [4]:
f1 = lambda x: 2 * x[0]**2 + x[1]**2
df1 = lambda x: np.array([4 * x[0], 2 * x[1]])
g1 = lambda x: np.array([4 * x[0]**2 + x[1]**2 - 2,
                        4 * x[0] + x[1] + 1])
dg1 = lambda x: np.array([[8 * x[0], 2 * x[1]],
                         [4, 1]]).T
x_sol = [-2 / 9, -1 / 9]
xi = 0.5
eta = 0.5
phi = 1.5
nu = 0.7578
x0 = [-1 / 3, -1]
m = 2
np.set_printoptions(precision=10, suppress=True, linewidth=np.nan)
print(' i step t[i]\tx[i]\t\t\t\tf(x[i])\t  error')
FDIPA(f1, df1, g1, dg1, x0, 1e-5, xi, eta, phi, nu, m)


 i step t[i]	x[i]				f(x[i])	  error
 1 0.5742608400 [-0.5019126593 -0.5612570257] 0.818842084 0.516189128
 2 0.5742608400 [-0.534974293  -0.2115320151] 0.617140782 0.558792563
 3 1.0000000000 [-0.4858518513 -0.1276460335] 0.488397553 0.102636239
 4 1.0000000000 [-0.4160473554 -0.1668050409] 0.374014726 0.077126870
 5 1.0000000000 [-0.3595439143 -0.1632885367] 0.285206799 0.056740111
 6 1.0000000000 [-0.3186817517 -0.154288741 ] 0.226921133 0.042219218
 7 1.0000000000 [-0.291573539  -0.1426705311] 0.190385138 0.029792807
 8 1.0000000000 [-0.2725392417 -0.1358389054] 0.167007485 0.020364325
 9 1.0000000000 [-0.2597279938 -0.1287020756] 0.151481486 0.014751855
10 1.0000000000 [-0.250065384  -0.1254354219] 0.140799438 0.010237579
11 1.0000000000 [-0.2435270585 -0.1209677983] 0.133244065 0.007945716
12 1.0000000000 [-0.2381652321 -0.1197181096] 0.127777781 0.005516009
13 1.0000000000 [-0.2346966394 -0.1165934708] 0.123759063 0.004678102
14 1.0000000000 [-0.2315066009 -0.116503832 ] 0.12076

array([-0.2222213196, -0.111114726 ])

#### Examen Final
##### 4. Usando el método FDIPA aproxime la solución de los siguientes problemas con restricción de desigualdad

\begin{align*}
\min&\quad 2x_1 + x_2 - 3x_1^2 - 2x_2^2 - 4x_1x_2\\
\text{s.t.}&\quad x_1+x_2-9\le 0\\
&2x_1-x_2\le 0\\
&-x_1\le 0\\
&-x_2\le 0
\end{align*}

In [5]:
f2 = lambda x: 2 * x[0] + x[1] - 3 * x[0]**2 - 2 * x[1]**2 - 4 * x[0] * x[1]
df2 = lambda x: np.array([2 - 6 * x[0] - 4 * x[1], 1 - 4 * x[1] - 4 * x[0]])
g2 = lambda x: np.array([x[0] + x[1] - 9,
                        2 * x[0] - x[1],
                        -x[0],
                        -x[1]])
dg2 = lambda x: np.array([[1, 1.],
                         [2, -1],
                         [-1, 0],
                         [0, -1]]).T
x0 = np.array([0.25, 1])
x_sol = [-2 / 9, -1 / 9]
xi = 0.5
eta = 0.5
phi = 1.5
nu = 0.7578
m = 4
np.set_printoptions(precision=10, suppress=True, linewidth=np.nan)
print(' i step t[i]\tx[i]\t\t\t\tf(x[i])\t\terror')
FDIPA(f2, df2, g2, dg2, x0, 1e-5, xi, eta, phi, nu, m)


 i step t[i]	x[i]				f(x[i])		error
 1 0.7578000000 [1.8766125487 5.9612950266] -116.672744852 1.849047071
 2 0.1087518886 [1.9486251594 7.0306932459] -154.125514496 14.742276854
 3 0.0473260884 [1.9777463795 7.0181208348] -154.789121021 0.806163023
 4 0.0624519509 [2.0113979828 6.9878177069] -155.006874232 0.730471995
 5 1.0000000000 [2.5368581678 6.4546040107] -156.600113230 0.748641232
 6 0.0624519509 [2.5656312936 6.434192778 ] -157.010675210 0.617301927
 7 1.0000000000 [2.989088048 5.989895875] -158.210879968 0.616874334
 8 0.0473260884 [2.9898426081 6.0078479459] -158.868496268 0.562085867
 9 0.0271775193 [2.9909340972 6.0089080861] -158.949229145 0.059803583
10 0.0271775193 [2.9913794986 6.0086034989] -158.956376720 0.020075346
11 0.5742608400 [2.999429524  6.0005686412] -158.997083729 0.019949836
12 0.0271775193 [2.9994558801 6.0005440417] -158.997276959 0.001327241
13 0.1893771627 [2.9996273669 6.000372632 ] -158.998136934 0.001280907
14 0.5742608400 [2.9999834157 6.000015943 

array([2.9999994754, 6.0000005242])

\begin{align*}
\min&\quad 100{(x_2-x_1^2)}^2 + {(1-x_1)}^2\\
\text{s.t.}&\quad 1-x_1x_2\le 0\\
&-x_1-x_2^2\le 0\\
&x_1-\frac{1}{2}\le 0\\
\end{align*}

In [6]:
f3 = lambda x: 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2
df3 = lambda x: np.array([400 * (x[0]**2 - x[1]) * x[0] + 2 * (x[0] - 1),
                         200 * (x[1] - x[0]**2)])
g3 = lambda x: np.array([1 - x[0] * x[1],
                        -x[0] - x[1]**2,
                        x[0] - 0.5])
dg3 = lambda x: np.array([[-x[1], -x[0]],
                         [-1, -2 * x[1]],
                         [1, 0]]).T
x0 = np.array([0.49, 3])
xi = 0.5
eta = 0.5
phi = 1.5
nu = 0.7578
m = 3
np.set_printoptions(precision=10, suppress=True, linewidth=np.nan)
print(' i step t[i]\tx[i]\t\t\t\tf(x[i])\t\terror')
FDIPA(f3, df3, g3, dg3, x0, 1e-5, xi, eta, phi, nu, m)


 i step t[i]	x[i]				f(x[i])		error
 1 0.0051468015 [0.3856905787 2.5952603369] 598.915124924 109.974694447
 2 0.0009746867 [0.4838321127 2.6092437646] 564.400300926 115.267178496
 3 0.0051468015 [0.4380135918 2.291766119 ] 441.278118755 95.976394402
 4 0.0016972891 [0.4992396913 2.2486253456] 400.004829167 99.522789205
 5 0.0029556065 [0.4733253009 2.1313965556] 364.079494510 61.034363314
 6 0.0016972891 [0.4994550439 2.0788697807] 334.926264513 77.330731057
 7 0.0022397586 [0.4927670998 2.0325463628] 320.569540112 34.702500422
 8 0.0009746867 [0.4986674736 2.0163146205] 312.708350178 37.762018550
 9 0.0012862057 [0.4997100976 2.0066542902] 308.935550070 14.743936155
10 0.0016972891 [0.4997802565 2.0016181525] 307.143804247 5.553628409
11 0.0009746867 [0.4999821046 2.0006519758] 306.734517576 2.053874434
12 0.0016972891 [0.4999669723 2.0001617472] 306.568207678 0.529418039
13 0.0007386176 [0.4999932904 2.0000738764] 306.528212444 0.258064063
14 0.0012862057 [0.4999999749 2.0000216505]

array([0.5         , 2.0000000052])